In [ ]:
!which python

In [ ]:
# !pip install cmake==3.27.0 pyarrow==16.1.0 datasets transformers accelerate sagemaker boto3 --upgrade --no-cache-dir

In [ ]:
# !pip install peft

In [ ]:
# !pip install -U bitsandbytes

In [1]:
import boto3

In [2]:
s3=boto3.client("s3")

In [3]:
response=s3.list_objects_v2(Bucket="llm-finetune-dataset-girish", Prefix="datasets/")

In [4]:
for obj in response.get("Contents", []):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2025, 11, 13, 11, 6, 5, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 13, 11, 58, 37, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [5]:
for obj in response.get("Contents", []):
    print(obj["Key"])

datasets/
datasets/pharma_instruction_data.csv


In [7]:
dataset_path = "s3://llm-finetune-dataset-girish/datasets/pharma_instruction_data.csv/"

In [8]:
import pyarrow
import datasets
import transformers
import accelerate
print(pyarrow.__version__)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


22.0.0


In [9]:
import cmake
print(cmake.__version__)

3.27.0


In [10]:
from datasets import load_dataset

In [11]:
# dataset = load_dataset("csv", data_files={"train":dataset_path})
dataset = load_dataset("csv",data_files={"train":dataset_path},split="train")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:298: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [12]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})

In [13]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})


In [14]:
print(dataset[0])

{'instruction': 'Explain the mechanism of action of Metformin.', 'input': None, 'output': 'Metformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'}


In [15]:
!pip install transformers accelerate sagemaker

In [16]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 33.1 MB/s  0:00:00


In [17]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 155.6 MB/s  0:00:00 eta 0:00:01


In [18]:
!pip install s3fs

In [19]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    return {"text": prompt}

In [20]:
dataset = dataset.map(format_example)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [21]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5
})

In [22]:
dataset['text'][0]

'### Instruction:\nExplain the mechanism of action of Metformin.\n### Input:\nNone\n### Response:\nMetformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [24]:
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [27]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [28]:
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [29]:
tokenized = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [30]:
tokenized

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})

In [33]:
tokenized['input_ids']

Column([[1, 835, 2799, 4080, 29901, 13, 9544, 7420, 278, 13336, 310, 3158, 310, 4737, 689, 262, 29889, 13, 2277, 29937, 10567, 29901, 13, 8516, 13, 2277, 29937, 13291, 29901, 13, 10095, 689, 262, 5039, 1078, 319, 3580, 29899, 11236, 630, 26823, 19015, 559, 313, 19297, 29968, 511, 607, 16415, 3144, 1682, 852, 318, 415, 1296, 322, 9950, 1017, 29899, 562, 333, 19100, 333, 362, 1550, 297, 6335, 11407, 540, 29886, 2454, 3144, 29884, 535, 29872, 6352, 6656, 29892, 27999, 5224, 292, 10416, 3144, 1682, 852, 29889, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [34]:
tokenized['attention_mask']

Column([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
from peft import LoraConfig, get_peft_model, TaskType

In [35]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

RuntimeError: unable to mmap 4400216536 bytes from file </home/ec2-user/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/59f6f375b26bde864a6ca194a9a3044570490064/model.safetensors>: Cannot allocate memory (12)

In [36]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [37]:
model = get_peft_model(model, lora_config)

NameError: name 'model' is not defined

In [38]:
args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [39]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
)

NameError: name 'model' is not defined

In [40]:
trainer.train()

NameError: name 'trainer' is not defined

In [41]:
model_path = "/content/tinyllama-instruction/checkpoint-3"

In [42]:
instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/tinyllama-instruction/checkpoint-3'. Use `repo_type` argument if needed.

In [43]:
prompt = "Explain the mechanism of action of Metformin."

In [44]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

RuntimeError: No CUDA GPUs are available

In [45]:

outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

NameError: name 'instruction_model' is not defined

In [46]:

print("\nModel Output:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Model Output:



NameError: name 'outputs' is not defined